# Aplicación de la Regresión Logística para el Análisis de sentimientos

Adaptado por http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

En esta sección se entrenó una regresión logística para clasificar opiniones de un dataset de 50K IMDb  recolectado por Maas el. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

La base de datos consiste en 50,000 opiniones de películas del original "entrenamiento" y "testeo". Las etiquetas son binarias y contienen 25,000 comentarios positivos y 25,000 comentarios negativos.

In [3]:
# Importamos la librería Panda
import pandas as pd

# Y leemos el archivo desde la hoja de cálculo.
df = pd.read_csv('shuffled_movie_data.csv')

#revisamos el contenido de los 5 últimos elementos con tail()
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


Let us shuffle the class labels.

In [4]:
# Importamos la librería Numpy
import numpy as np
#reordenamos el contenido de la hoja de cálculo
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)

<br>
<br>

## Preprocesamiento de la Información

Se utilizó la función 'tokenizer' para convertir el texto en una matriz de palabras así como la conversión a minúsculas y el filtrado de símbolos que no tienen trascendencia .

In [5]:
# Importamos la librería nltk para tratar con cadenas de texto
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

#stop=stopwords.words('english')

Con la función 'stopwords.words('english')'extraimos palabras importantes y los almacenamos en matrices con los nombres 'stop', 'negativos' y 'pronom'

In [6]:
stop=[u'what', u'which', u'who', u'whom', u'this', u'that', u"that'll", u'these', u'those', u'am', u'is', u'are', 
      u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing',
      u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by',
      u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above',
      u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further',
      u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', 
      u'more', u'most', u'other', u'some', u'such',  u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', 
      u't', u'can', u'will', u'just', u'should', u"should've", u'now', u'd', u'll', u'm', u'o', u're', u've', u'y', u'ma']

In [7]:
negativos=[ u'no', u'nor', u'not', u"don't", u'aren', u"aren't", u'couldn', u"couldn't", u'didn', u"didn't", u'doesn',
      u"doesn't", u'hadn', u"hadn't", u'hasn', u"hasn't", u'haven', u"haven't", u'isn', u"isn't", u'mightn',
      u"mightn't", u'mustn', u"mustn't", u'needn', u"needn't", u'shan', u"shan't", u'shouldn', u"shouldn't", u'wasn',
      u"wasn't", u'weren', u"weren't", u'won', u"won't", u'wouldn', u"wouldn't"]

In [8]:
pronom=[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u"you're", u"you've", u"you'll",
        u"you'd", u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u"she's",
        u'her', u'hers', u'herself', u'it', u"it's", u'its', u'itself', u'they', u'them', u'their', u'theirs',
        u'themselves']

In [9]:
#Código del 'tokenizer'
porter = PorterStemmer()
exclamation = '!'
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('!', ' ! ', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+' + exclamation + ']', ' ', text) + ' '.join(emoticons).replace('-', '')
    text=text.split()
    #text = [w for w in text.split() if w not in stop]
    #tokenized = [porter.stem(w) for w in text]
    return text

Veamos una prueba:

In [10]:
tokenizer('This :) is a <a> test! :-)</br>')

['This', ':)', 'is', 'a', 'test', '!', ':-):)', ':)']

# Regresión Logística

Primero se crea un buffer para trabajar con el texto de la hoja de cálculo

In [11]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

Ejercicio 1: Definir nuevas características accorde con https://web.stanford.edu/~jurafsky/slp3/5.pdf

## Extracción de las características del texto

Estas son las funciones que extraen las características del texto

Esta función cuenta la cantidad de palabras positivas de un texto.

In [12]:
with open("positive-words.txt") as word_file:
    positive_words = set(word.strip().lower() for word in word_file)

def is_positive_word(word):
    return word.lower() in positive_words

Esta función cuenta la cantidad de palabras negativas de un texto.

In [13]:
with open("negative-words.txt") as word_file:
    negative_words = set(word.strip().lower() for word in word_file)

def is_negative_word(word):
    return word.lower() in negative_words

Esta función cuenta la cantidad de palabras negativas como : no, not, nor, can't, etc...

In [14]:
def is_neg_word(word):
    if(unicode(word.lower(), "utf-8") in negativos):
        return True
    return False

Esta función cuenta la cantidad de pronombres de una texto.

In [15]:
def is_first_second_p(word):
    if(unicode(word.lower(), "utf-8") in pronom):
        return True
    return False

Esta función cuenta la cantidad de signos de interrogación dentro del texto

In [16]:
def is_sign_word(word):
    return word=="!"

Esta función tiene como entrada un texto y lo convierte en una matriz [ x0, x1, x2, x3, x4, x5]

In [17]:
def transformFeatures(unTexto): #ingresar buffer
    x=np.array([0,0,0,0,0,0])
    texto=np.array(unTexto)
    for i in range(texto.size):
        if(is_positive_word(texto[i])):
            x[0]+=1
        if(is_negative_word(texto[i])):
            x[1]+=1
        if(is_neg_word(texto[i])):
            x[2]+=1
        if(is_first_second_p(texto[i])):
            x[3]+=1
        if(is_sign_word(texto[i])):
            x[4]+=1
    x[5]=np.log(texto.size)
    
    return x

En esta función se extrae todo los textos de la hoja de calculo y los almacena en un array.

In [18]:
def get_batch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text  = next(doc_stream)
        texto = tokenizer(text[0])
        label = int(text[1])
        docs.append(transformFeatures(texto))
        y.append(label)
    return np.array(docs), np.array(y)

Tomamos los textos de la hoja de cálculo y realizamos la conversión a una matriz con, en este caso, las 6 variables y características.

In [19]:
flujo=stream_docs(path='shuffled_movie_data.csv')

In [20]:
#esta parte del código toma aproximadamente 1 o 2 min
Xx, Yy=get_batch(flujo, 50000)
Xx.shape

(50000, 6)

In [21]:
Yy.shape

(50000,)

Ejercicio 2: Implementación de la Regresión Logística usando Regularización acorde con https://web.stanford.edu/~jurafsky/slp3/5.pdf

## Regresión Logística con Regularización

Funcion para trabajar con la regresion logistica

In [22]:
def regresionLog(Xtraining,Ytraining,inicio,final,iteraciones, alpha):
    #Donde:
    # Xtraining: Elementos de entrada de entrenamiento
    # Ytraining: Elementos de salida de entrenamiento
    # inicio   : index del primer elemento en Xtraining y Ytraining
    # final    : index del elemento después del último de Xtraining y Ytraining
    # iteraciones : Cantidad de repeticiones del proceso.
    # alpha    : El número que multiplica al gradiente.
    
    Ww=10*np.array(np.random.rand(6))-5 #Los pesos inicializados con valores aleatorios
    b=0 #El bias
    beta=0.999 #importante variable en la Regresión Logística con Regularización
    for j in range(iteraciones):
        for i in range(inicio,final):
            Cc=1.0/(1+np.exp(-np.matmul(Xtraining[i],Ww)-b)) - Ytraining[i]
            b=beta*b-alpha*Cc
            Ww=beta*Ww-alpha*Cc*Xtraining[i] #Esta es la expresión iterativa de de la Regresión Logística con Regularización
    return Ww,b

Ejemplo: Realizamos una iteración

In [30]:
Ww,b=regresionLog(Xx,Yy,0,45000,50,0.0001)
print(Ww,b)

(array([ 0.04898685, -0.04621436, -0.0156574 ,  0.00149933, -0.00394275,
       -0.00234278]), -0.0007441865295391508)


Y luego otra para corroborar su convergencia

In [31]:
Ww,b=regresionLog(Xx,Yy,0,45000,100,0.0001)
print(Ww,b)

(array([ 0.04898685, -0.04621436, -0.0156574 ,  0.00149933, -0.00394275,
       -0.00234278]), -0.0007441865295391508)


El valor de $\alpha$ fue escogido de forma empirica, en base a ensayo

Creamos funciones que nos permitan medir la precisión del proceso, primero creamos la función SIGMOIDE
$$ F(z)=\frac{1}{1+\exp(-z)}$$

In [32]:
def funcionResult(W,b,X,inicio,fin):
    Y=np.round(1.0/(1+np.exp(-np.matmul(X[inicio:fin],Ww)-b)))
    return Y

Esta función se encarga de calcular el porcentaje de acierto del porceso.

In [33]:
def calculoAcc(Ytest,Ypred,size):
    acierto=0
    for i in range(size):
        if(Ytest[i]==Ypred[i]):
            acierto+=1
    return (1.0*acierto)/size

In [34]:
Y_test=funcionResult(Ww,b,Xx,0,50000)

In [35]:
calculoAcc(Y_test,Yy,50000)

0.7178

Con una tasa de acierto del 71,78%

Se realizo el entrenamiento cruzado donde se formaron 10 grupos de 5000 textos y después se corroboraron con todo el universo de pruebas 

In [36]:
for i in range(10):
    print("Test numero",i+1)
    Ww,b=regresionLog(Xx,Yy,5000*i,5000*i+5000,500,0.00001)
    print(Ww,b)
    Y_test=funcionResult(Ww,b,Xx,0,50000)
    print(calculoAcc(Y_test,Yy,50000))

('Test numero', 1)
(array([ 7.09826827e-03, -5.35924595e-03, -1.97437050e-03,  2.10011696e-03,
       -5.69098750e-05, -2.25075978e-04]), -5.180512391145822e-05)
0.60354
('Test numero', 2)
(array([ 6.94571692e-03, -5.82104865e-03, -1.82751308e-03,  1.28016559e-03,
       -7.61435821e-04, -4.82729307e-05]), -1.1554800416377122e-05)
0.65808
('Test numero', 3)
(array([ 0.00678149, -0.00685713, -0.00232508,  0.00115488, -0.00016672,
       -0.00024902]), -6.516498576348736e-05)
0.69678
('Test numero', 4)
(array([ 0.00667454, -0.00623989, -0.00234839, -0.00116093, -0.00034825,
       -0.00021956]), -3.328029209493929e-05)
0.70122
('Test numero', 5)
(array([ 5.65463852e-03, -6.91844043e-03, -2.64180712e-03, -2.04342983e-03,
       -6.22819745e-06, -6.13961814e-04]), -0.00010293213477328954)
0.6194
('Test numero', 6)
(array([ 7.15345486e-03, -5.23805676e-03, -1.88995372e-03,  2.00590800e-03,
       -2.98022070e-04,  5.52617048e-05]), 2.0549625039643275e-05)
0.5969
('Test numero', 7)
(array([ 

En este caso vemos que la mejor prueba es aquella que tiene un acierto de un 71,9% correspondiente a los pesos:
$\theta =$ [ 0.00636925, -0.00668156, -0.00197957, -0.00053378, -0.00045964,
       -0.00029726]
y un $bias=$-8.081830650110261e-05

Observación: Los valores pueden variar por cada entrenamiento

<br>
<br>